# TitleAbstract Reviewer with RIS files

In this notebook, we will demostrate the use of `TitleAbstractReviewer` agents when your input data is in the '.ris' format.

## Setting up the notebook

High-level configs

In [1]:
%reload_ext autoreload
%autoreload 2

from dotenv import load_dotenv

# Load environment variables from .env file. Adjust the path to the .env file as needed.
load_dotenv(dotenv_path='../.env')

# Enable asyncio in Jupyter
import asyncio
import nest_asyncio

nest_asyncio.apply()

#  Add the package to the path (required if you are running this notebook from the examples folder)
import sys
sys.path.append('../../')


Import required packages

In [3]:
import pandas as pd 

from lattereview.providers import LiteLLMProvider, OpenAIProvider
from lattereview.agents import TitleAbstractReviewer
from lattereview.workflows import ReviewWorkflow
from lattereview.utils.data_handler import ris_to_dataframe

## Data

First, let's load a dummy dataset of a few aritcles that we downloaded from PubMed. We can use the `ris_to_dataframe` function to convert the RIS file to show its data in a pandas DataFrame, but the workflow class can handle the RIS file directly.


In [7]:
ris_data_path = 'data.ris'

# Convert the RIS file to a pandas DataFrame
df_data = await ris_to_dataframe('data.ris')
df_data


,type,authors,author_address,title,T2,JF,volume,issue,start_page,year,date,abstract,issn,doi,url,keywords,publisher,C2
0,JOUR,"Binh, Le Nguyen; Nhu, Nguyen Thanh; Nhi, Pham ...","College of Medicine, Taipei Medical University...",Impact of deep learning on pediatric elbow fra...,Eur. J. Trauma Emerg. Surg.,European journal of trauma and emergency surge...,51,1,115,2025,2025/2/20,OBJECTIVES: Pediatric elbow fractures are a co...,1863-9933,10.1007/s00068-025-02779-w,http://dx.doi.org/10.1007/s00068-025-02779-w; ...,Convolutional neural network; Deep learning; M...,NaN,NaN
1,JOUR,"Zhang, Subo; Zhu, Zhitao; Yu, Zhenfei; Sun, Ha...","Department of Medical Imaging, The Second Peop...",Effectiveness of AI for enhancing computed tom...,J. Med. Internet Res.,Journal of medical internet research,27,NaN,e66622,2025,2025/2/27,BACKGROUND: Artificial intelligence (AI) prese...,1439-4456,10.2196/66622,http://dx.doi.org/10.2196/66622; https://www.n...,artificial intelligence; computed tomography; ...,NaN,NaN
2,JOUR,"Wu, Qian; Guo, Hui; Li, Ruihan; Han, Jinhuan","Department of Medical Imaging Center, The Four...",Deep learning and machine learning in CT-based...,Int. J. Med. Inform.,International journal of medical informatics,196,105812,105812,2025,2025/4,BACKGROUND: With advancements in medical techn...,1386-5056,10.1016/j.ijmedinf.2025.105812,http://dx.doi.org/10.1016/j.ijmedinf.2025.1058...,Artificial Intelligence; Chronic Obstructive P...,Elsevier BV,NaN
3,JOUR,"Sadeghi, Termeh Sarrafan; Ourang, Seyed Amirho...","Dentofacial Deformities Research center, Resea...",Performance of artificial intelligence on cerv...,BMC Oral Health,BMC oral health,25,1,187,2025,2025/2/5,BACKGROUND: Artificial intelligence (AI) metho...,1472-6831,10.1186/s12903-025-05482-9,http://dx.doi.org/10.1186/s12903-025-05482-9; ...,Artificial intelligence; Cervical vertebrae; C...,Springer Science and Business Media LLC,PMC11796225
4,JOUR,"Abbott, Lyndon P; Saikia, Ankita; Anthonappa, ...","Paediatric Dentistry, UWA Dental School, The U...",Artificial Intelligence platforms in dental ca...,J. Evid. Based. Dent. Pract.,The journal of evidence-based dental practice,25,1,102077,2025,2025/3,OBJECTIVES: To assess Artificial Intelligence ...,1532-3382,10.1016/j.jebdp.2024.102077,http://dx.doi.org/10.1016/j.jebdp.2024.102077;...,Artificial intelligence; Deep learning; Dental...,Elsevier BV,NaN
5,JOUR,"Tao, Huimin; Hui, Xu; Zhang, Zhihong; Zhu, Ron...","The First Clinical Medical College of Gansu, U...",Accuracy of artificial intelligence in detecti...,BMC Cancer,BMC cancer,25,1,286,2025,2025/2/18,BACKGROUND: Bone metastases (BM) represent a p...,1471-2407,10.1186/s12885-025-13631-0,http://dx.doi.org/10.1186/s12885-025-13631-0; ...,Artificial intelligence; Bone metastases; Diag...,NaN,PMC11837447
6,JOUR,"Zhang, Zhihui; Zhang, Yijing; Wang, He; Lei, M...",Department of Radiology and Tianjin Key Labora...,Resting-state network alterations in depressio...,Psychol. Med.,Psychological medicine,55,NaN,e63,2025,2025/2/26,BACKGROUND: Depression has been linked to disr...,0033-2917,10.1017/S0033291725000303,http://dx.doi.org/10.1017/S0033291725000303; h...,depression; fMRI; functional connectivity; neu...,NaN,NaN
7,JOUR,"Rokhshad, Rata; Banakar, Morteza; Shobeiri, Pa...","Loma Linda School of Dentistry, University of ...",Artificial intelligence in early childhood car...,Pediatr. Dent.,Pediatric dentistry,46,6,385-394,2024,2024/11/15,Purpose: To conduct a systematic review of art...,0164-1263,NaN,https://www.ncbi.nlm.nih.gov/pubmed/40036840 E...,NaN,NaN,NaN
8,JOUR,"Tarighatnia, Ali; Amanzadeh, Masoud; Hamedan, ...","Department of Medical Physics, School of Medic...",Deep learning-based evaluation of panoramic ra...,BMC Med. Imaging,BMC medical imaging,25,1,86,2025,2025/3/12,BACKGROUND: Osteoporosis is a complex conditio...,1471-2342,10.1186/s12880-025-01626-z,http://dx.doi.org/10.1186/s12880-025-01626-z; ...,Deep learning; Diagnosis; OPG; Osteoporosis; P...,NaN,PMC11905531
9,JOUR,"Mubon

Let's then write some inclusion and exclusion criteria to screen these papers.

In [8]:
inclusion_criteria = "The study must involve AI and CT scans."

## Review

Then, we will define three reviewers (two juniors and one senior) that will go through the dataset and look for those items that meet all the inclusion criteria and does not violate any of the exclusion criteria. Each agent return either of the following scores:

- 1 means absolutely to exclude.
- 2 means better to exclude.
- 3 Not sure if to include or exclude.
- 4 means better to include.
- 5 means absolutely to include.

In [13]:
Agent1 = TitleAbstractReviewer(
    provider=LiteLLMProvider(model="gemini/gemini-1.5-flash"),
    name="Agent1",
    backstory="a PhD researcher in biology and computer science",
    inclusion_criteria = inclusion_criteria,
    max_concurrent_requests=30, 
    model_args={"max_tokens": 200, "temperature": 0.1},
)

Agent2 = TitleAbstractReviewer(
    provider=LiteLLMProvider(model="o3-mini"), 
    name="Agent2",
    backstory="a PhD researcher in biology and computer science",
    inclusion_criteria = inclusion_criteria,
    max_concurrent_requests=30, 
    model_args={'reasoning_effort': 'low'},
)

Agent3 = TitleAbstractReviewer(
    provider=LiteLLMProvider(model="o3-mini"),
    name="Agent3",
    backstory="a senior MD-PhD researcher with years of experience in conducting systematic reviews in radiology and deep learning",
    inclusion_criteria = inclusion_criteria,
    max_concurrent_requests=30, 
    model_args={'reasoning_effort': 'high'},
    additional_context="""
    Two PhD reviewers have already reviewed this article and disagree on how to evaluate it. You can read their evaluation above.
    """
)

Finally, let's create a review workflow and put our agents to work!

In [16]:
def filter_func(row):
    score1 = int(row["round-A_Agent1_output"]["evaluation"])
    score2 = int(row["round-A_Agent2_output"]["evaluation"])
    if score1 != score2:
        if score1 >= 4 and score2 >= 4:
            return False
        if score1 >= 3 or score2 >= 3:
            return True
    elif score1 == score2 == 3:
        return True
    return False

async def review(df, sample_size=None):
    
    title_abs_review = ReviewWorkflow(
        workflow_schema=[
                {
                    "round": 'A',
                    "reviewers": [Agent1, Agent2],
                    "text_inputs": ["title", "abstract"]
                },
                {
                    "round": 'B',
                    "reviewers": [Agent3],
                    "text_inputs": ["title", "abstract", "round-A_Agent1_output", "round-A_Agent2_output"],
                    "filter": filter_func
                }
            ]
        )

    if sample_size:
        df = df.sample(sample_size)

    updated_df = await title_abs_review(df)
    total_cost = title_abs_review.get_total_cost()
    print(f"\n====== Finished reviewing ======\n")
    print(f"\nTotal cost: {total_cost}")
    print("-" * 100)
    return updated_df


updated_df = asyncio.run(review(ris_data_path))

Converting RIS file: data.ris

====== Starting review round A (1/2) ======

Processing 10 eligible rows


['round: A', 'reviewer_name: Agent1'] -                     2025-03-16 13:44:16: 100%|██████████| 10/10 [00:00<00:00, 12.74it/s]


The following columns are present in the dataframe at the end of Agent1's reivew in round A: ['type', 'authors', 'author_address', 'title', 'T2', 'JF', 'volume', 'issue', 'start_page', 'year', 'date', 'abstract', 'issn', 'doi', 'url', 'keywords', 'publisher', 'C2', 'round-A_Agent1_output', 'round-A_Agent1_reasoning', 'round-A_Agent1_evaluation']


['round: A', 'reviewer_name: Agent2'] -                     2025-03-16 13:44:17: 100%|██████████| 10/10 [00:03<00:00,  2.84it/s]

The following columns are present in the dataframe at the end of Agent2's reivew in round A: ['type', 'authors', 'author_address', 'title', 'T2', 'JF', 'volume', 'issue', 'start_page', 'year', 'date', 'abstract', 'issn', 'doi', 'url', 'keywords', 'publisher', 'C2', 'round-A_Agent1_output', 'round-A_Agent1_reasoning', 'round-A_Agent1_evaluation', 'round-A_Agent2_output', 'round-A_Agent2_reasoning', 'round-A_Agent2_evaluation']

====== Starting review round B (2/2) ======

Skipping review round B - no eligible rows

====== Finished reviewing ======


Total cost: 0.0014901
----------------------------------------------------------------------------------------------------


In [17]:
updated_df

,type,authors,author_address,title,T2,JF,volume,issue,start_page,year,...,url,keywords,publisher,C2,round-A_Agent1_output,round-A_Agent1_reasoning,round-A_Agent1_evaluation,round-A_Agent2_output,round-A_Agent2_reasoning,round-A_Agent2_evaluation
0,JOUR,"Binh, Le Nguyen; Nhu, Nguyen Thanh; Nhi, Pham ...","College of Medicine, Taipei Medical University...",Impact of deep learning on pediatric elbow fra...,Eur. J. Trauma Emerg. Surg.,European journal of trauma and emergency surge...,51,1,115,2025,...,http://dx.doi.org/10.1007/s00068-025-02779-w; ...,Convolutional neural network; Deep learning; M...,NaN,NaN,"{'evaluation': 1, 'reasoning': 'The abstract d...",The abstract does not mention the use of CT sc...,1,{'reasoning': 'The study involves AI but does ...,The study involves AI but does not utilize CT ...,1
1,JOUR,"Zhang, Subo; Zhu, Zhitao; Yu, Zhenfei; Sun, Ha...","Department of Medical Imaging, The Second Peop...",Effectiveness of AI for enhancing computed tom...,J. Med. Internet Res.,Journal of medical internet research,27,NaN,e66622,2025,...,http://dx.doi.org/10.2196/66622; https://www.n...,artificial intelligence; computed tomography; ...,NaN,NaN,"{'evaluation': 5, 'reasoning': 'The study clea...","The study clearly involves AI and CT scans, me...",5,{'reasoning': 'The study directly involves AI ...,The study directly involves AI and CT scans by...,5
2,JOUR,"Wu, Qian; Guo, Hui; Li, Ruihan; Han, Jinhuan","Department of Medical Imaging Center, The Four...",Deep learning and machine learning in CT-based...,Int. J. Med. Inform.,International journal of medical informatics,196,105812,105812,2025,...,http://dx.doi.org/10.1016/j.ijmedinf.2025.1058...,Artificial Intelligence; Chronic Obstructive P...,Elsevier BV,NaN,"{'evaluation': 5, 'reasoning': 'The study invo...",The study involves AI (deep learning and machi...,5,{'reasoning': 'The study clearly involves the ...,The study clearly involves the use of AI (both...,5
3,JOUR,"Sadeghi, Termeh Sarrafan; Ourang, Seyed Amirho...","Dentofacial Deformities Research center, Resea...",Performance of artificial intelligence on cerv...,BMC Oral Health,BMC oral health,25,1,187,2025,...,http://dx.doi.org/10.1186/s12903-025-05482-9; ...,Artificial intelligence; Cervical vertebrae; C...,Springer Science and Business Media LLC,PMC11796225,"{'evaluation': 1, 'reasoning': 'The study uses...","The study uses radiographs, not CT scans, thus...",1,{'reasoning': 'The study involves AI for cervi...,The study involves AI for cervical vertebral m...,1
4,JOUR,"Abbott, Lyndon P; Saikia, Ankita; Anthonappa, ...","Paediatric Dentistry, UWA Dental School, The U...",Artificial Intelligence platforms in dental ca...,J. Evid. Based. Dent. Pract.,The journal of evidence-based dental practice,25,1,102077,2025,...,http://dx.doi.org/10.1016/j.jebdp.2024.102077;...,Artificial intelligence; Deep learning; Dental...,Elsevier BV,NaN,"{'evaluation': 1, 'reasoning': 'The study does...","The study does not involve CT scans, thus fail...",1,{'reasoning': 'The study does not involve CT s...,The study does not involve CT scans as require...,1
5,JOUR,"Tao, Huimin; Hui, Xu; Zhang, Zhihong; Zhu, Ron...","The First Clinical Medical College of Gansu, U...",Accuracy of artificial intelligence in detecti...,BMC Cancer,BMC cancer,25,1,286,2025,...,http://dx.doi.org/10.1186/s12885-025-13631-0; ...,Artificial intelligence; Bone metastases; Diag...,NaN,PMC11837447,"{'evaluation': 4, 'reasoning': 'The abstract e...",The abstract explicitly mentions the use of AI...,4,{'reasoning': 'The study meets the inclusion c...,The study meets the inclusion criteria by focu...,5
6,JOUR,"Zhang, Zhihui; Zhang, Yijing; Wang, He; Lei, M...",Department of Radiology and Tianjin Key Labora...,Resting-state network alterations in depressio...,Psychol. Med.,Psychological medicine,55,NaN,e63,2025,...,http://dx.doi.org/10.1017/S0033291725000303; h...,depression; fMRI; functional connectivity; neu...,NaN,NaN,"{'evaluation': 1, 'reasoning': 'The study does...","T